In [278]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)

## Dataset : Cinema Tickets

In [279]:
ticket = pd.read_csv("Dataset\movie_daily.csv")
ticket["date"] = pd.to_datetime(ticket["date"])
ticket.drop(["tickets_out","ticket_use","month","quarter","day"], axis = 1, inplace = True)
display(ticket.head())

,film_code,cinema_code,total_sales,tickets_sold,show_time,occu_perc,ticket_price,capacity,date
0,1492,304,3900000,26,4,4.26,150000.0,610.328638,2018-05-05
1,1492,352,3360000,42,5,8.08,80000.0,519.801980,2018-05-05
2,1492,489,2560000,32,4,20.00,80000.0,160.000000,2018-05-05
3,1492,429,1200000,12,1,11.01,100000.0,108.991826,2018-05-05
4,1492,524,1200000,15,3,16.67,80000.0,89.982004,2018-05-05


In [280]:
display(ticket.isna().sum())
ticket.dropna(inplace = True)
assert ticket.isna().sum().sum() == 0

film_code         0
cinema_code       0
total_sales       0
tickets_sold      0
show_time         0
occu_perc       125
ticket_price      0
capacity        125
date              0
dtype: int64

In [281]:
# Select top 10 movies to study
ticket = ticket[(ticket.film_code != 1484) & (ticket.film_code != 1482)]
films = ticket.film_code.value_counts().head(10).index.to_list()
print("Cinema to study:", films)
ticket = ticket.loc[ticket.film_code.isin(films)]

Cinema to study: [1554, 1493, 1481, 1556, 1483, 1485, 1494, 1498, 1499, 1576]


In [282]:
ticket_agg = ticket.groupby(["cinema_code", "film_code"]).agg({"total_sales": ["sum", "mean"], 
                                                               "tickets_sold": ["sum"], 
                                                               "show_time": ["sum"], 
                                                               "capacity": ["sum", "mean"], 
                                                               "date": ["min", "max"]}).reset_index()

ticket_agg.columns = ["cinema_code", "film_code", "total_sales_sum", "total_sales_mean", "tickets_sold_sum", "show_time_sum", "capacity_sum", "capacity_daily_mean", "released_date", "last_date"]
ticket_agg["days_in_theater"] = (ticket_agg.last_date - ticket_agg.released_date).dt.days + 1
ticket_agg["capacity_single_mean"] = ticket_agg.capacity_sum/ticket_agg.show_time_sum

display(ticket_agg.head())

,cinema_code,film_code,total_sales_sum,total_sales_mean,tickets_sold_sum,show_time_sum,capacity_sum,capacity_daily_mean,released_date,last_date,days_in_theater,capacity_single_mean
0,32,1483,4224010000,5.557908e+07,51011,1138,534893.838713,7038.076825,2018-03-14,2018-05-29,77,470.029735
1,32,1554,3511299998,4.620132e+07,41919,978,459683.953180,6048.473068,2018-07-04,2018-10-02,91,470.024492
2,33,1483,711000000,9.233766e+06,8253,378,135128.318466,1754.913227,2018-03-14,2018-05-30,78,357.482324
3,33,1494,504950000,5.804023e+06,6661,497,177488.212334,2040.094395,2018-05-02,2018-07-31,91,357.119140
4,33,1554,1120030000,1.623232e+07,12669,413,148693.048888,2154.971723,2018-07-04,2018-09-11,70,360.031595


In [283]:
ticket_all = ticket.merge(ticket_agg, how = "left", on = ["film_code", "cinema_code"])
ticket_all["nth_day_from_release"] = pd.to_datetime(ticket_all["date"]) - pd.to_datetime(ticket_all["released_date"])
ticket_all["nth_day_from_release"] = ticket_all["nth_day_from_release"].dt.days + 1
display(ticket_all.head())

,film_code,cinema_code,total_sales,tickets_sold,show_time,occu_perc,ticket_price,capacity,date,total_sales_sum,total_sales_mean,tickets_sold_sum,show_time_sum,capacity_sum,capacity_daily_mean,released_date,last_date,days_in_theater,capacity_single_mean,nth_day_from_release
0,1498,448,73969998,520,6,45.61,142249.996154,1140.100855,2018-07-25,2529459976,3.372613e+07,20522,359,68379.297366,911.723965,2018-06-27,2018-09-11,77,190.471580,29
1,1498,304,37650000,256,10,15.06,147070.312500,1699.867198,2018-07-25,1975300000,2.469125e+07,15645,437,73393.149530,917.414369,2018-06-27,2018-10-07,103,167.947711,29
2,1498,71,32400000,216,6,15.06,150000.000000,1434.262948,2018-07-25,1238340000,2.579875e+07,9810,197,47084.295501,980.922823,2018-06-27,2018-08-14,49,239.006576,29
3,1498,89,30060000,276,8,43.13,108913.043478,639.925806,2018-07-25,907270000,1.106427e+07,8604,350,35193.583330,429.190041,2018-06-27,2018-09-25,91,100.553095,29
4,1498,141,19950000,133,3,50.96,150000.000000,260.989011,2018-07-25,536210000,6.619877e+06,4725,209,20948.334174,258.621410,2018-06-29,2018-09-23,87,100.231264,27


In [284]:
df = ticket_all.groupby(["film_code", "nth_day_from_release"]).agg({"tickets_sold":"mean", "show_time":"mean"}).reset_index()
fig = px.histogram(df, x="nth_day_from_release", y="tickets_sold", color='film_code', barmode='group', histfunc='avg', height = 600, hover_data=df.columns)

fig.update_layout(title = "Distribution of Tickets Sold",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Numbers of Tickets Sold",
                  legend_title = "Film Code")

fig.show()

In [285]:
df = ticket_all.groupby(["film_code", "nth_day_from_release"]).agg({"occu_perc":"mean"}).reset_index()
fig = px.histogram(df, x="nth_day_from_release", y="occu_perc", color='film_code', barmode='group', histfunc='avg', height = 600)

fig.update_layout(title = "Distribution of Theater Usage",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Theater Usage Percentage",
                  legend_title = "Film Code")

fig.show()

## Dataset : Box Office

In [286]:
bo = pd.read_csv("Datasets/cinemaTicket_Ref.csv")
bo.drop(["Distr", "index"], axis = 1, inplace = True)
bo["date"] = pd.to_datetime(bo["date"])
display(bo.head())

,Movie,Gross,date
0,Despicable Me 2,6845130,2013-07-16
1,Grown Ups 2,5273521,2013-07-16
2,Pacific Rim,4416340,2013-07-16
3,The Heat,2175635,2013-07-16
4,Monsters University,1931131,2013-07-16


In [287]:
display(bo.isna().sum())
bo.dropna(inplace = True)
assert bo.isna().sum().sum() == 0

Movie    97
Gross     0
date      0
dtype: int64

In [288]:
bo_agg = bo.groupby(["Movie"]).agg({"date": ["min","max"],
                                    "Gross": ["sum"]})
bo_agg.columns = ["released_date", "last_date", "total_gross"]
bo_agg["days_in_theater"] = (bo_agg.last_date - bo_agg.released_date).dt.days + 1
bo_agg.reset_index(inplace = True)
display(bo_agg.head())

,Movie,released_date,last_date,total_gross,days_in_theater
0,10 Cloverfield Lane,2016-03-10,2016-06-02,73882999,85
1,102 Not Out,2018-05-04,2018-06-03,1339909,31
2,12 O’Clock Boys,2014-02-13,2014-02-13,10705,1
3,12 Strong,2018-01-18,2018-03-29,46400164,71
4,12 Years a Slave,2013-10-18,2014-04-24,56634802,189


In [289]:
most_recent = bo_agg.sort_values(by = "released_date", ascending = False).head(50)
most_recent = most_recent["Movie"].to_list()

In [290]:
bo_all = bo.merge(bo_agg, how = "left", on = "Movie")
bo_all["nth_day_from_release"] = pd.to_datetime(bo_all["date"]) - pd.to_datetime(bo_all["released_date"])
bo_all["nth_day_from_release"] = bo_all["nth_day_from_release"].dt.days + 1
bo_all = bo_all.loc[bo_all.Movie.isin(most_recent)]
display(bo_all.head())

,Movie,Gross,date,released_date,last_date,total_gross,days_in_theater,nth_day_from_release
112144,Pray: The Story of Patric…,10402,2020-10-09,2020-10-09,2020-12-20,71735,73,1
112173,Pray: The Story of Patric…,6353,2020-10-10,2020-10-09,2020-12-20,71735,73,2
112196,Pray: The Story of Patric…,7642,2020-10-11,2020-10-09,2020-12-20,71735,73,3
112219,Pray: The Story of Patric…,3263,2020-10-12,2020-10-09,2020-12-20,71735,73,4
112243,Pray: The Story of Patric…,2297,2020-10-13,2020-10-09,2020-12-20,71735,73,5


In [291]:
df = bo_agg[(bo_agg.days_in_theater <= 365) & (bo_agg.days_in_theater >= 7)]
df["weeks_in_theater"] = df["days_in_theater"].apply(lambda x: math.ceil(x/7))
fig = px.histogram(df, x = "weeks_in_theater", marginal = "box")
fig.update_layout(title = "Distribution of Weeks in Theater",
                  xaxis_title = "Weeks in Theater",
                  yaxis_title = "Numbers of Movies")
fig.show()

/var/folders/p1/pm0sdphs6hg08b36xt9g88ch0000gn/T/ipykernel_3107/2817570476.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [292]:
df = bo_all[bo_all.nth_day_from_release <= 180]
df = df.groupby(["Movie","nth_day_from_release"]).agg({"Gross":"mean"}).reset_index()
fig = px.histogram(df, x="nth_day_from_release", y="Gross", barmode='group', histfunc='avg', height = 600, hover_data=df.columns)

fig.update_layout(title = "Distribution of Box Office",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Box Office")

fig.show()

In [293]:
df = bo_all[bo_all.nth_day_from_release <= 180]
df["weeks_from_release"] = df["nth_day_from_release"].apply(lambda x: math.ceil(x/7))
df = df.groupby(["Movie","weeks_from_release"]).agg({"Gross":"sum"}).reset_index()
fig = px.histogram(df, x="weeks_from_release", y="Gross", barmode='group', color = "Movie", histfunc='avg', height = 600, hover_data=df.columns)

fig.update_layout(title = "Distribution of Box Office",
                  xaxis_title = "Weeks after Movies are Released",
                  yaxis_title = "Box Office")

fig.show()

In [337]:
df_2022 = pd.read_excel('Datasets/imdb_data.xlsx', sheet_name='2022')
df_2021 = pd.read_excel('Datasets/imdb_data.xlsx', sheet_name='2021')
df_2020 = pd.read_excel('Datasets/imdb_data.xlsx', sheet_name='2020')
df = pd.concat([df_2022, df_2021, df_2020])
df = df.reset_index(drop=True)
df = df[['Date', 'Top 10 Gross', '%± YD', '#1 Release', 'Gross']]
df.columns = ['date', 'top_10_gross', 'day_change', 'n1_release', 'gross']

In [338]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df.date.dt.year.astype('str')
df['weekday'] = df.date.dt.day_name()

In [339]:
df = df.merge(df.groupby('n1_release').date.min(), how='left', on='n1_release', suffixes=('','_start'))
df['date_lag'] = (df.date - df.date_start).dt.days + 1
#df['date_lag'] = (df.date - df.date_start).dt.days
#df['date_lag_adj'] = df.date_lag + 0.00001


In [340]:
#df = df.merge(df.query('date_lag == 0')[['n1_release', 'gross']], how='left', on='n1_release', suffixes=('','_start'))
#df['gross_lag'] = df.gross / df.gross_start

df = df.merge(df.query('date_lag == 1')[['n1_release', 'gross']], how='left', on='n1_release', suffixes=('','_start'))
df['gross_lag'] = df.gross / df.gross_start

In [341]:
outliers = ['The Wretched', 'The Croods: A New Age', 'Becky']
df = df.query(f'n1_release not in {outliers}')


In [342]:
df.head()

,date,top_10_gross,day_change,n1_release,gross,year,weekday,date_start,date_lag,gross_start,gross_lag
0,2022-12-31,27962493,0.013,Avatar: The Way of Water,18053159,2022,Saturday,2022-12-16,16,53200270,0.339343
1,2022-12-30,37900613,0.215,Avatar: The Way of Water,24836835,2022,Friday,2022-12-16,15,53200270,0.466855
2,2022-12-29,31204428,-0.021,Avatar: The Way of Water,20117061,2022,Thursday,2022-12-16,14,53200270,0.378138
3,2022-12-28,31866473,-0.147,Avatar: The Way of Water,20582014,2022,Wednesday,2022-12-16,13,53200270,0.386878
4,2022-12-27,37343124,-0.205,Avatar: The Way of Water,24128503,2022,Tuesday,2022-12-16,12,53200270,0.453541


In [346]:
dftmp = df[df.date_lag <= 60].copy()
dftmp["week"] = dftmp.date_lag.apply(lambda x: int((x-1)/7))
fig = px.histogram(dftmp, x="date_lag", y="gross_lag", barmode='group', color = "n1_release", histfunc='avg', height = 600, hover_data=df.columns)

fig.update_layout(title = "Trends of Box Office",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Box Office")

fig.show()

In [388]:
fig = px.scatter(dftmp, 
                 x='date_lag', y='gross_lag', color='gross_lag', 
                 hover_data='n1_release',
                 trendline="lowess", trendline_scope="overall", 
                 #trendline="ols", trendline_options=dict(log_x=True), trendline_scope="overall", 
                 color_continuous_scale= px.colors.diverging.Portland,
                 trendline_color_override="magenta", height = 600, width = 1200)

fig.update_layout(legend=dict(yanchor = "top", y = 0.98, xanchor = "right", x = 0.99), 
coloraxis_colorbar=dict(title = "Scaled Box Office", yanchor = "top", y = 1, ticks="outside"))

fig.update_layout(title = "Trends of Box Office",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Box Office")
fig.show()

In [348]:
fig = px.line(dftmp, x="date_lag", y="gross_lag", color = "n1_release", height = 600, hover_data=df.columns)
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title = "Trends of Box Office",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Box Office")

fig.show()

In [389]:
df.head()

,date,top_10_gross,day_change,n1_release,gross,year,weekday,date_start,date_lag,gross_start,gross_lag
0,2022-12-31,27962493,0.013,Avatar: The Way of Water,18053159,2022,Saturday,2022-12-16,16,53200270,0.339343
1,2022-12-30,37900613,0.215,Avatar: The Way of Water,24836835,2022,Friday,2022-12-16,15,53200270,0.466855
2,2022-12-29,31204428,-0.021,Avatar: The Way of Water,20117061,2022,Thursday,2022-12-16,14,53200270,0.378138
3,2022-12-28,31866473,-0.147,Avatar: The Way of Water,20582014,2022,Wednesday,2022-12-16,13,53200270,0.386878
4,2022-12-27,37343124,-0.205,Avatar: The Way of Water,24128503,2022,Tuesday,2022-12-16,12,53200270,0.453541


In [391]:
df.n1_release.value_counts()

Spider-Man: No Way Home                      43
Tenet                                        36
Black Panther: Wakanda Forever               34
Shang-Chi and the Legend of the Ten Rings    28
Top Gun: Maverick                            24
                                             ..
Everything Everywhere All at Once             1
Lightyear                                     1
Judas and the Black Messiah                   1
Unsubscribe                                   1
Monster Hunter                                1
Name: n1_release, Length: 103, dtype: int64

In [418]:
PRICE = 20 # Original price
M_DAY = 19 # Markdown starting day
def get_sales(row):
    if row["date_lag"] < M_DAY:
        return row["scaled_sales_og"]
    else: 
        return int(row["scaled_sales_og"] * 1.2)

df_tenet = df[df.n1_release == "Tenet"]
df_tenet["scaled_sales_og"] = df_tenet.top_10_gross.apply(lambda x: int(x/10000))
df_tenet["scaled_sales_new"] = df_tenet.apply(lambda row: get_sales(row), axis = 1)

fig = px.line(height = 600)
fig.add_scatter(x=df_tenet.date_lag, y=df_tenet.scaled_sales_new, name = "With Markdown")
fig.add_scatter(x=df_tenet.date_lag, y=df_tenet.scaled_sales_og, name = "No Markdown")

#fig.update_traces(line=dict(width=0.5))
fig.update_layout(title = "Scaled Sales Trend of Tenet",
                  xaxis_title = "Days after Tenet is Released",
                  yaxis_title = "Box Office")

fig.show()

/var/folders/p1/pm0sdphs6hg08b36xt9g88ch0000gn/T/ipykernel_3107/1911501972.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/p1/pm0sdphs6hg08b36xt9g88ch0000gn/T/ipykernel_3107/1911501972.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [438]:
df_comparison = pd.melt(df_tenet[["n1_release", "date_lag","scaled_sales_og","scaled_sales_new"]], id_vars = ["n1_release", "date_lag"], var_name = "policy", value_name = "sales",)
display(df_comparison)

,n1_release,date_lag,policy,sales
0,Tenet,36,scaled_sales_og,60
1,Tenet,35,scaled_sales_og,57
2,Tenet,34,scaled_sales_og,67
3,Tenet,33,scaled_sales_og,52
4,Tenet,32,scaled_sales_og,196
...,...,...,...,...
67,Tenet,5,scaled_sales_new,364
68,Tenet,4,scaled_sales_new,521
69,Tenet,3,scaled_sales_new,616
70,Tenet,2,scaled_sales_new,465


In [458]:
def get_rev(row):
    if row["policy"] == "scaled_sales_new" and row["date_lag"] >= M_DAY:
        return row["sales"] * PRICE * 0.9
    else: 
        return row["sales"] * PRICE

df_comparison["revenue"] = df_comparison.apply(lambda row: get_rev(row), axis = 1)

fig = px.ecdf(df_comparison, x="date_lag", y="revenue", color="policy", ecdfnorm=None, height = 600, width = 800, markers = True, lines = True)

newnames = {'scaled_sales_og':'Without Markdown', 'scaled_sales_new': 'With Markdown'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

fig.update_layout(title = "Cumulated Revenue based on Policy",
                  xaxis_title = "Days after Movies are Released",
                  yaxis_title = "Revenue")

fig.show()